In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
#Used headers/agent because the request was timed out and asking for an agent. 
#Using following code we can fake the agent.
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'}
response = requests.get("https://www.lahaus.mx/buscar?locations=area-metropolitana-monterrey",headers=headers)

In [ ]:
content = response.content
soup = BeautifulSoup(content,"html.parser")

In [ ]:
lahaus_res = soup.find_all("article",attrs={"class":"property-card-container border mb-lh-24 md:flex-row rounded-lg w-full cursor-pointer new-property border-lh-md-gray"})

In [ ]:
list_lh_res = list(lahaus_res)

In [ ]:
def create_newrow(cols_list,data_list):
       
    newrow_dict = {}
    for i,j in zip(cols_list,data_list):
        newrow_dict[i] = j
        
    return newrow_dict

def merge_df(newrow_dict,target_df):
    
    newrow_df = pd.DataFrame(newrow_dict, index=[len(target_df)])
    target_df = pd.concat([target_df,newrow_df])
    
    return target_df

df_lahaus = pd.DataFrame(columns = ['loc','name','desc','ammenities','categories','price_range','unit','unit_area','unit_price','bedrooms','bathrooms','parking_spots'])

for res in lahaus_res:
    new = True
    categories = []
    if (res.find("span",attrs={'class':'leading-8 text-12 lg:text-10 xl:text-12 rounded-sm text-lh-jungle-green bg-lh-caribbean-green px-lh-8 py-lh-4 mr-5 truncate'})) is not None:
        categories.append(res.find("span",attrs={'class':'leading-8 text-12 lg:text-10 xl:text-12 rounded-sm text-lh-jungle-green bg-lh-caribbean-green px-lh-8 py-lh-4 mr-5 truncate'}).text)
    if (res.find("span",attrs={'class':'leading-10 text-12 lg:text-10 xl:text-12 border rounded-sm text-lh-green-gray border-lh-gray px-lh-8 mr-5 truncate'})) is not None:
        categories.append(res.find("span",attrs={'class':'leading-10 text-12 lg:text-10 xl:text-12 border rounded-sm text-lh-green-gray border-lh-gray px-lh-8 mr-5 truncate'}).text)
    if (res.find("span",attrs={'class':'leading-8 text-12 lg:text-10 xl:text-12 rounded-sm text-success-800 bg-success-50 px-lh-8 py-lh-4 mr-5 truncate'})) is not None:
        categories.append(res.find("span",attrs={'class':'leading-8 text-12 lg:text-10 xl:text-12 rounded-sm text-success-800 bg-success-50 px-lh-8 py-lh-4 mr-5 truncate'}).text)
        #continue
    categories_con = "; ".join([str(i).strip() for i in categories])
    
    loc = (res.find("span",attrs={'class':'label-r-12 order-2 text-lh-green-gray mt-6 mb-4'}).text).strip()
    name = (res.find("a").contents[0]).strip()
    desc = (res.find("p",attrs={'class':'order-4 text-12 font-regular text-lh-dark-deep-blue mb-3 leading-7'}).text).strip()
    ammenities = (res.find("span",attrs={'class':'label-r-12 order-5 text-lh-green-gray mb-8'}).text).strip()
    price_range = (res.find("p",attrs={'class':'order-6 text-14 font-bold text-lh-dark-deep-blue'}).text).strip()
    
    if new:
        for u,unit in enumerate(res.find_all("li", attrs={'class':'p-lh-12 mr-lh-12 border rounded-lg border-lh-md-gray hover:bg-lh-extra-light-blue'})):
            
            unit_area = (unit.find("p",attrs={'class':'text-14 text-lh-dark-deep-blue text-lh-green-gray'}).text).strip()
            unit_price = (unit.find("p",attrs={'class':'small-r-14 text-lh-jungle-green mt-4'}).text).strip()
                                  
            for details in unit.find_all("div",attrs={'class':'unit-details-icons flex small-r-14 text-lh-green-gray'}):                
                              
                if (details.find("i",attrs={'class':'fa fa-real-estate-bedroom'})) is not None:
                    bedrooms = (details.find("span",attrs={'class':'mr-lh-16'}).text).strip()
                if (details.find("i",attrs={'class':'fa fa-real-estate-bathroom'})) is not None:
                    bathrooms = (details.find("span",attrs={'class':'mr-lh-16'}).text).strip()
                if (details.find("i",attrs={'class':'fa fa-real-estate-parking'})) is not None:
                    parking_spots = (details.find("span",attrs={'class':'mr-lh-16'}).text).strip()
                else:
                    parking_spots = 0
            
            #print([loc,name,desc,ammenities,categories_con,price_range,u+1,unit_area,unit_price,bedrooms,bathrooms,parking_spots])
            list_cols = ['loc','name','desc','ammenities','categories','price_range','unit','unit_area','unit_price','bedrooms','bathrooms','parking_spots']
            newrow_list = [loc,name,desc,ammenities,categories_con,price_range,u+1,unit_area,unit_price,bedrooms,bathrooms,parking_spots]            
            
            df_lahaus = merge_df(create_newrow(list_cols,newrow_list),df_lahaus)



In [248]:
df_lahaus

,loc,name,desc,ammenities,categories,price_range,unit,unit_area,unit_price,bedrooms,bathrooms,parking_spots
0,"Monterrey, Área Metropolitana Monterrey",Semillero Mediterráneo,Semillero Mediterráneo es tú mejor inversión. ...,Incluye: Piscina · Zonas Verdes · BBQ y más.,Desarrollo nuevo; Preventa,De 1.96 a $2.63 millones,1,59 ㎡,$2.63 millones,2,2,2
1,"Monterrey, Área Metropolitana Monterrey",Semillero Mediterráneo,Semillero Mediterráneo es tú mejor inversión. ...,Incluye: Piscina · Zonas Verdes · BBQ y más.,Desarrollo nuevo; Preventa,De 1.96 a $2.63 millones,2,49 ㎡,$2.61 millones,1,1,1
2,"Monterrey, Área Metropolitana Monterrey",Semillero Mediterráneo,Semillero Mediterráneo es tú mejor inversión. ...,Incluye: Piscina · Zonas Verdes · BBQ y más.,Desarrollo nuevo; Preventa,De 1.96 a $2.63 millones,3,50 ㎡,$2.48 millones,2,2,2
3,"Monterrey, Área Metropolitana Monterrey",Semillero Mediterráneo,Semillero Mediterráneo es tú mejor inversión. ...,Incluye: Piscina · Zonas Verdes · BBQ y más.,Desarrollo nuevo; Preventa,De 1.96 a $2.63 millones,4,48 ㎡,$2.44 millones,2,2,2
4,"Monterrey, Área Metropolitana Monterrey",Semillero Mediterráneo,Semillero Mediterráneo es tú mejor inversión. ...,Incluye: Piscina · Zonas Verdes · BBQ y más.,Desarrollo nuevo; Preventa,De 1.96 a $2.63 millones,5,41 ㎡,$2.3 millones,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
58,"Monterrey, Área Metropolitana Monterrey",Barrio W,Te presentamos nuestra segunda torre de depart...,Incluye: Asador · Sala de Juegos · Alberca y más.,Desarrollo nuevo; Preventa,De 1.65 a $4.5 millones,1,95 ㎡,$4.3 millones,3,3,3
59,"Monterrey, Área Metropolitana Monterrey",Barrio W,Te presentamos nuestra segunda torre de depart...,Incluye: Asador · Sala de Juegos · Alberca y más.,Desarrollo nuevo; Preventa,De 1.65 a $4.5 millones,2,98 ㎡,$4.1 millones,2,2,2
60,"Monterrey, Área Metropolitana Monterrey",Barrio W,Te presentamos nuestra segunda torre de depart...,Incluye: Asador · Sala de Juegos · Alberca y más.,Desarrollo nuevo; Preventa,De 1.65 a $4.5 millones,3,95 ㎡,$4.04 millones,3,3,3
61,"Monterrey, Área Metropolitana Monterrey",Barrio W,Te presentamos nuestra segunda torre de depart...,Incluye: Asador · Sala de Juegos · Alberca y más.,Desarrollo nuevo; Preventa,De 1.65 a $4.5 millones,4,82 ㎡,$3.55 millones,2,2,2
